In [269]:

"""
Loading packages 
"""


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sn

%matplotlib inline

plt.style.use('bmh')

import re, json, requests
from random import sample
import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import rc

from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, Activation, LSTM, SimpleRNN, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bert
from tqdm import tqdm
from tensorflow.keras import backend as K
import tensorflow as tf

print("TensorFlow Version:",tf.__version__)
print("Keras Version:",keras.__version__)
# Params for bert model and tokenization


TensorFlow Version: 2.3.0
Keras Version: 2.4.0


In [272]:
 """   
 Loading JSON imbalanced data set directly diretly from the GitHub 
 
 
"""
    
url = 'https://raw.githubusercontent.com/clinc/oos-eval/master/data/data_imbalanced.json'
 resp = requests.get(url)
 data = json.loads(resp.text)


In [273]:
"""
Cheking keys to understand what data is there
"""

data.keys()

dict_keys(['oos_val', 'val', 'train', 'oos_test', 'test', 'oos_train'])

In [274]:

print("Sample from train")
print(" ")
#print(data['val'][0:10])
print(sample(data['train'],10))

print(" ")
print("size of train "+str(len(data['train'])))


train_arr_np = np.asarray(data['train'])

train_pd= pd.DataFrame(train_arr_np, columns=['sentence', 'intent']) 

Sample from train
 
[["i was at trader joe's trying to buy groceries and my card got declined", 'card_declined'], ['will you roll a 6 sided dice', 'roll_dice'], ["i just traveled on american airlines flight ab123 out of dallas i can't find my luggage", 'lost_luggage'], ['i am going to be traveling in turkey, are there any alerts', 'travel_alert'], ['can you slow your speaking down', 'change_speed'], ['can you tell me which timezone is cambodia in', 'timezone'], ['any travel advisories for portugal', 'travel_alert'], ['give me all contents of my shopping list', 'shopping_list'], ['tell me what the traffic is like on the way to phoenix', 'traffic'], ['how do i say hello in french', 'translate']]
 
size of train 10525


In [275]:

print("Sample from validation")
print(" ")
#print(data['val'][0:10])
print(sample(data['val'],10))

print(" ")
print("size of validation "+str(len(data['val'])))

val_arr_np = np.asarray(data['val'])
val_pd= pd.DataFrame(val_arr_np, columns=['sentence', 'intent']) 

Sample from validation
 
[['how much air pressure is in my tires', 'tire_pressure'], ['forget reservation at oyster bar for tonight', 'cancel_reservation'], ["i'm not sure how to jump start a car; can you help me", 'jump_start'], ['in a fried egg, how many calories are there', 'calories'], ['remind me to call my mom', 'reminder_update'], ['what does idiosyncratic mean', 'definition'], ['can you find me an uber and have it here at 7pm', 'uber'], ['tell me something to make me laugh', 'tell_joke'], ['how old were you on your most recent birthday', 'how_old_are_you'], ['can you please turn your volume down', 'change_volume']]
 
size of validation 3000


In [276]:

print("Sample from test")
print(" ")
#print(data['val'][0:10])
print(sample(data['test'],10))

print(" ")
print("size of test "+str(len(data['test'])))

test_arr_np = np.asarray(data['test'])
test_pd= pd.DataFrame(test_arr_np, columns=['sentence', 'intent']) 

Sample from test
 
[['when is my meeting with jack for', 'meeting_schedule'], ["i'm looking for a meal that's from the asian cuisine", 'meal_suggestion'], ['my account at the bank needs to be frozen', 'freeze_account'], ['tell me about the pets you have', 'do_you_have_pets'], ['are there meetings from 1 to 5 pm', 'meeting_schedule'], ['state recent transaction', 'transactions'], ['what time is it in california', 'time'], ['what does serendipity mean', 'definition'], ['will my fico score decrease if i increase my student loan debt', 'improve_credit_score'], ['i want to know if i will be safe if i travel to europe', 'travel_alert']]
 
size of test 4500


In [277]:
type_counts_train = train_pd['intent'].value_counts()

print(type_counts_train)

"""

THIS SHOWS THE SENTENCES PER INTENT ARE IMBALANCED.

TOTAL OF 150 INTENT CLASSES

"""



tire_change           100
current_location      100
order                 100
traffic               100
gas_type              100
                     ... 
application_status     25
freeze_account         25
bill_balance           25
redeem_rewards         25
report_fraud           25
Name: intent, Length: 150, dtype: int64


'\n\nTHIS SHOWS THE SENTENCES PER INTENT ARE IMBALANCED.\n\nTOTAL OF 150 INTENT CLASSES\n\n'

In [278]:
type_counts_val = val_pd['intent'].value_counts()

print(type_counts_val)

tire_change         20
oil_change_how      20
reset_settings      20
direct_deposit      20
next_holiday        20
                    ..
order_status        20
transfer            20
gas                 20
schedule_meeting    20
fun_fact            20
Name: intent, Length: 150, dtype: int64


In [279]:
type_counts = test_pd['intent'].value_counts()

print(type_counts)

tire_change         30
oil_change_how      30
reset_settings      30
direct_deposit      30
next_holiday        30
                    ..
order_status        30
transfer            30
gas                 30
schedule_meeting    30
fun_fact            30
Name: intent, Length: 150, dtype: int64


In [280]:
print("Sample from oos_train")
print(" ")
#print(data['val'][0:10])
print(sample(data['oos_train'],10))

print(" ")
print("size of oos_train "+str(len(data['oos_train'])))


train_arr_np_oos = np.asarray(data['oos_train'])

train_pd_oos= pd.DataFrame(train_arr_np_oos, columns=['sentence', 'intent']) 

Sample from oos_train
 
[['how can i get involved in restaurant marketing', 'oos'], ['can you tell me something about the nfl', 'oos'], ['what animals have alpha males', 'oos'], ['what time does the louvre open', 'oos'], ['how much is an overdraft fee for bank', 'oos'], ['how expensive is an apple share', 'oos'], ["who's toledo's starting point guard for the next game", 'oos'], ['when was nintendo created', 'oos'], ['when was toyota created', 'oos'], ['are any justice league movies coming out', 'oos']]
 
size of oos_train 100


In [281]:
print("Sample from oos_val")
print(" ")
#print(data['val'][0:10])
print(sample(data['oos_val'],10))

print(" ")
print("size of oos_val "+str(len(data['oos_val'])))


val_arr_np_oos = np.asarray(data['oos_val'])

val_pd_oos= pd.DataFrame(val_arr_np_oos, columns=['sentence', 'intent']) 

Sample from oos_val
 
[['do ducks sleep', 'oos'], ['how do you know when your dog is ill', 'oos'], ['what university in the united states offers the best biomedical engineering program', 'oos'], ['how many firewalls should i have and what type', 'oos'], ['what is the purpose or benefit of a cash advance', 'oos'], ['what is the friendliest breed', 'oos'], ['what is naval engineering', 'oos'], ['show me the channel guide', 'oos'], ['king of acid', 'oos'], ['are there any new companies to look out for', 'oos']]
 
size of oos_val 100


In [282]:
print("Sample from oos_test")
print(" ")
#print(data['val'][0:10])
print(sample(data['oos_test'],10))

print(" ")
print("size of oos_test "+str(len(data['oos_test'])))


test_arr_np_oos = np.asarray(data['oos_test'])

test_pd_oos= pd.DataFrame(test_arr_np_oos, columns=['sentence', 'intent']) 

Sample from oos_test
 
[['look up daily dog kennels in my area', 'oos'], ['how to be millionare', 'oos'], ["what do you do all day while i'm at work", 'oos'], ['where does jelly come from', 'oos'], ['change color contrast', 'oos'], ['how do i get stains out of the carpet', 'oos'], ['can you tell me what the best places are to look for a job online', 'oos'], ['show me a list of top rated airlines', 'oos'], ['get me a list of universities that provide full scholarships in arizona', 'oos'], ['are we allowed to wash our cars during the drought', 'oos']]
 
size of oos_test 1000


In [283]:

"""
Finding if any nan values in any of the dataset sentences or intent column


"""

print("train nan check: "+str(train_pd.isna().sum()))
print(" ")
print("test nan check: "+str(test_pd.isna().sum()))
print(" ")
print("val nan check: "+str(val_pd.isna().sum()))
print(" ")
print("train_oos nan check: "+str(train_pd_oos.isna().sum()))
print(" ")
print("test_oos nan check: "+str(test_pd_oos.isna().sum()))
print(" ")
print("val_oos nan check: "+str(val_pd_oos.isna().sum()))

train nan check: sentence    0
intent      0
dtype: int64
 
test nan check: sentence    0
intent      0
dtype: int64
 
val nan check: sentence    0
intent      0
dtype: int64
 
train_oos nan check: sentence    0
intent      0
dtype: int64
 
test_oos nan check: sentence    0
intent      0
dtype: int64
 
val_oos nan check: sentence    0
intent      0
dtype: int64


In [284]:
"""
AS PER THE OOS-TRAIN PROCEDURE, WE WILL TAKE OOS AS 151th Intent. For that, we combine 

the respevtive train, with train-oss, test with test-oos and val with val-oos
"""

'\nAS PER THE OOS-TRAIN PROCEDURE, WE WILL TAKE OOS AS 151th Intent. For that, we combine \n\nthe respevtive train, with train-oss, test with test-oos and val with val-oos\n'

In [285]:
"""
  Train Conacte 
"""

frames_train=[train_pd, train_pd_oos]

train=pd.concat(frames_train)

In [286]:
"""
  valid Conacte 
"""



frames_val=[val_pd, val_pd_oos]

val=pd.concat(frames_val)

In [287]:
"""
  test Conacte 
"""



frames_test=[test_pd, test_pd_oos]

test=pd.concat(frames_test)

In [288]:
"""
Downloading Bert Simple moodel

"""


!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-10-17 19:39:46--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.196.144, 172.217.167.80, 142.250.66.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.196.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: 'uncased_L-12_H-768_A-12.zip.1’

uncased_L-12_H-768_ 100%[===================>] 388.84M  53.8MB/s    in 15s     

2020-10-17 19:40:02 (26.8 MB/s) - 'uncased_L-12_H-768_A-12.zip.1’ saved [407727028/407727028]



In [289]:
"""
Unzipping the model

"""


!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [290]:
"""
Making a directory with a name model

"""




os.makedirs("model", exist_ok=True)

In [291]:
"""
Moving  the dowmloaded BERT model in model directory

"""



!mv uncased_L-12_H-768_A-12/ model

mv: cannot move 'uncased_L-12_H-768_A-12/' to 'model/uncased_L-12_H-768_A-12': Directory not empty


In [292]:
"""
Assigning paths for model loading with trainied weights and saving the model
after fine tuning

"""


bert_model_name="uncased_L-12_H-768_A-12"

bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

In [293]:

"""
This class use BERT tokenzer to create tokens for each stence in all

three data sets. It also appends shorter sentences to the maximum length sentence

which is 33 and adds start and end of the sentence token

"""




class IntentDetectionData:
  DATA_COLUMN = "sentence"
  LABEL_COLUMN = "intent"

  def __init__(self, train, val,test, tokenizer: FullTokenizer, classes, max_seq_len=192):
    self.tokenizer = tokenizer
    self.max_seq_len = 0
    self.classes = classes
    
    ((self.train_x, self.train_y),(self.val_x, self.val_y), (self.test_x, self.test_y)) = map(self._prepare, [train,val,  test])

    print("max seq_len", self.max_seq_len)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x, self.val_x, self.test_x = map(self._pad, [self.train_x, self.val_x, self.test_x])

  def _prepare(self, df):
    x, y = [], []
    
    for _, row in tqdm(df.iterrows()):
      text, label = row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMN]
      tokens = self.tokenizer.tokenize(text)
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len = max(self.max_seq_len, len(token_ids))
      x.append(token_ids)
      y.append(self.classes.index(label))

    return np.array(x), np.array(y)

  def _pad(self, ids):
    x = []
    for input_ids in ids:
      input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)

In [294]:


tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

In [295]:
"""
LOading BERT model and adding one more fully connected layer with drop out

at end output

"""





def create_model(max_seq_len, bert_ckpt_file):

  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params, name="bert")
        
  input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
  bert_output = bert(input_ids)

  print("bert shape", bert_output.shape)

  cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
  cls_out = keras.layers.Dropout(0.5)(cls_out)
  logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
  logits = keras.layers.Dropout(0.5)(logits)
  logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)

  model = keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))

  load_stock_weights(bert, bert_ckpt_file)
        
  return model

In [296]:




classes = train.intent.unique().tolist()

data = IntentDetectionData(train, val,test, tokenizer, classes, max_seq_len=128)

10625it [00:01, 6439.75it/s]
<ipython-input-293-36d7e455bb91>:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(x), np.array(y)
3100it [00:01, 2525.22it/s]
5500it [00:00, 6441.65it/s]


max seq_len 33


In [297]:
classes

['direct_deposit',
 'carry_on',
 'whisper_mode',
 'text',
 'recipe',
 'smart_home',
 'who_do_you_work_for',
 'rewards_balance',
 'restaurant_reservation',
 'travel_notification',
 'update_playlist',
 'change_volume',
 'routing',
 'mpg',
 'bill_balance',
 'do_you_have_pets',
 'cook_time',
 'what_song',
 'new_card',
 'todo_list_update',
 'traffic',
 'next_song',
 'where_are_you_from',
 'tire_change',
 'bill_due',
 'greeting',
 'taxes',
 'lost_luggage',
 'change_accent',
 'todo_list',
 'last_maintenance',
 'make_call',
 'gas_type',
 'cancel_reservation',
 'schedule_meeting',
 'find_phone',
 'insurance_change',
 'improve_credit_score',
 'travel_suggestion',
 'roll_dice',
 'repeat',
 'play_music',
 'are_you_a_bot',
 'sync_device',
 'calendar',
 'insurance',
 'international_visa',
 'freeze_account',
 'shopping_list',
 'oil_change_when',
 'share_location',
 'what_can_i_ask_you',
 'plug_type',
 'vaccines',
 'payday',
 'application_status',
 'next_holiday',
 'tell_joke',
 'ingredient_substituti

In [298]:
data.train_x.shape

(10625, 33)

In [299]:
data.train_y.shape

(10625,)

In [300]:
data.test_x.shape

(5500, 33)

In [301]:
data.val_x.shape

(3100, 33)

In [302]:
data.max_seq_len

33

In [303]:
"""
weights for each class intent because of imbalanaced data

"""




from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(data.train_y),
                                                 data.train_y)

/home/abdul/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass classes=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150], y=[  0   0   0 ... 150 150 150] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [304]:
class_weights

array([1.40728477, 0.70364238, 0.70364238, 1.40728477, 0.93818985,
       0.70364238, 0.93818985, 2.81456954, 0.93818985, 0.70364238,
       0.70364238, 0.70364238, 2.81456954, 0.70364238, 2.81456954,
       0.93818985, 0.93818985, 0.70364238, 2.81456954, 0.70364238,
       0.70364238, 0.70364238, 0.93818985, 0.70364238, 2.81456954,
       0.93818985, 1.40728477, 0.70364238, 0.70364238, 0.70364238,
       0.70364238, 1.40728477, 0.70364238, 0.93818985, 1.40728477,
       1.40728477, 1.40728477, 2.81456954, 0.70364238, 1.40728477,
       0.70364238, 0.70364238, 0.70364238, 0.70364238, 0.70364238,
       1.40728477, 0.70364238, 2.81456954, 0.70364238, 0.70364238,
       1.40728477, 0.93818985, 0.70364238, 0.70364238, 1.40728477,
       2.81456954, 1.40728477, 0.93818985, 0.93818985, 0.70364238,
       0.93818985, 0.70364238, 1.40728477, 1.40728477, 1.40728477,
       0.93818985, 2.81456954, 0.93818985, 1.40728477, 2.81456954,
       0.70364238, 0.70364238, 2.81456954, 0.93818985, 0.70364

In [305]:
class_weights = dict(enumerate(class_weights))

In [306]:
class_weights

{0: 1.4072847682119205,
 1: 0.7036423841059603,
 2: 0.7036423841059603,
 3: 1.4072847682119205,
 4: 0.9381898454746137,
 5: 0.7036423841059603,
 6: 0.9381898454746137,
 7: 2.814569536423841,
 8: 0.9381898454746137,
 9: 0.7036423841059603,
 10: 0.7036423841059603,
 11: 0.7036423841059603,
 12: 2.814569536423841,
 13: 0.7036423841059603,
 14: 2.814569536423841,
 15: 0.9381898454746137,
 16: 0.9381898454746137,
 17: 0.7036423841059603,
 18: 2.814569536423841,
 19: 0.7036423841059603,
 20: 0.7036423841059603,
 21: 0.7036423841059603,
 22: 0.9381898454746137,
 23: 0.7036423841059603,
 24: 2.814569536423841,
 25: 0.9381898454746137,
 26: 1.4072847682119205,
 27: 0.7036423841059603,
 28: 0.7036423841059603,
 29: 0.7036423841059603,
 30: 0.7036423841059603,
 31: 1.4072847682119205,
 32: 0.7036423841059603,
 33: 0.9381898454746137,
 34: 1.4072847682119205,
 35: 1.4072847682119205,
 36: 1.4072847682119205,
 37: 2.814569536423841,
 38: 0.7036423841059603,
 39: 1.4072847682119205,
 40: 0.703642384

In [307]:
model = create_model(data.max_seq_len, bert_ckpt_file)

bert shape (None, 33, 768)
Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7ff1d7290a30> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [308]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 33)]              0         
_________________________________________________________________
bert (BertModelLayer)        (None, 33, 768)           108890112 
_________________________________________________________________
lambda_2 (Lambda)            (None, 768)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 768)               590592    
_________________________________________________________________
dropout_5 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 151)              

In [309]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [310]:
log_dir = "log/intent_detection/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(
  x=data.train_x, 
  y=data.train_y,
  validation_data=(data.val_x,data.val_y),
                              class_weight=class_weights,
  batch_size=16,
  shuffle=True,
  epochs=100,
  callbacks=[tensorboard_callback]
)

Epoch 1/100
665/665 [==============================] - 125s 188ms/step - loss: 5.0170 - acc: 0.0084 - val_loss: 5.0149 - val_acc: 0.0110
Epoch 2/100
665/665 [==============================] - 123s 186ms/step - loss: 5.0100 - acc: 0.0235 - val_loss: 5.0029 - val_acc: 0.0219
Epoch 3/100
665/665 [==============================] - 125s 189ms/step - loss: 4.9803 - acc: 0.0728 - val_loss: 4.9576 - val_acc: 0.0703
Epoch 4/100
665/665 [==============================] - 125s 188ms/step - loss: 4.9349 - acc: 0.1275 - val_loss: 4.9065 - val_acc: 0.1268
Epoch 5/100
665/665 [==============================] - 125s 187ms/step - loss: 4.8825 - acc: 0.1884 - val_loss: 4.8427 - val_acc: 0.2048
Epoch 6/100
665/665 [==============================] - 126s 190ms/step - loss: 4.8123 - acc: 0.2726 - val_loss: 4.7812 - val_acc: 0.2565
Epoch 7/100
665/665 [==============================] - 127s 192ms/step - loss: 4.7499 - acc: 0.3180 - val_loss: 4.7432 - val_acc: 0.2916
Epoch 8/100
665/665 [====================

665/665 [==============================] - 125s 188ms/step - loss: 4.0578 - acc: 0.9656 - val_loss: 4.1894 - val_acc: 0.8439
Epoch 59/100
665/665 [==============================] - 125s 188ms/step - loss: 4.0571 - acc: 0.9669 - val_loss: 4.1861 - val_acc: 0.8458
Epoch 60/100
665/665 [==============================] - 127s 192ms/step - loss: 4.0550 - acc: 0.9686 - val_loss: 4.1893 - val_acc: 0.8413
Epoch 61/100
665/665 [==============================] - 128s 192ms/step - loss: 4.0541 - acc: 0.9693 - val_loss: 4.1899 - val_acc: 0.8387
Epoch 62/100
665/665 [==============================] - 127s 191ms/step - loss: 4.0549 - acc: 0.9680 - val_loss: 4.1868 - val_acc: 0.8455
Epoch 63/100
665/665 [==============================] - 129s 194ms/step - loss: 4.0556 - acc: 0.9680 - val_loss: 4.1932 - val_acc: 0.8371
Epoch 64/100
665/665 [==============================] - 128s 192ms/step - loss: 4.0552 - acc: 0.9684 - val_loss: 4.1911 - val_acc: 0.8384
Epoch 65/100
665/665 [=========================

In [319]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [320]:
%tensorboard --logdir log

Reusing TensorBoard on port 6006 (pid 289584), started 6:46:35 ago. (Use '!kill 289584' to kill it.)

In [313]:
_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)

print("train acc", train_acc)
print("test acc", test_acc)

172/172 [==============================] - 10s 57ms/step - loss: 4.2635 - acc: 0.7707
train acc 0.9939764738082886
test acc 0.770727276802063


In [314]:
_, train_acc = model.evaluate(data.train_x, data.train_y)
_, val_acc = model.evaluate(data.val_x, data.val_y)

print("train acc", train_acc)
print("val acc", val_acc)

97/97 [==============================] - 6s 58ms/step - loss: 4.1799 - acc: 0.8490
train acc 0.9939764738082886
val acc 0.8490322828292847


In [315]:
y_pred_test = model.predict(data.test_x).argmax(axis=-1)

In [316]:
print(classification_report(data.test_y, y_pred_test, target_names=classes))

                           precision    recall  f1-score   support

           direct_deposit       0.81      0.97      0.88        30
                 carry_on       1.00      0.87      0.93        30
             whisper_mode       0.97      1.00      0.98        30
                     text       0.79      0.90      0.84        30
                   recipe       0.30      0.83      0.44        30
               smart_home       0.68      0.90      0.77        30
      who_do_you_work_for       0.64      0.53      0.58        30
          rewards_balance       0.69      0.97      0.81        30
   restaurant_reservation       0.63      0.87      0.73        30
      travel_notification       0.90      0.90      0.90        30
          update_playlist       0.97      0.93      0.95        30
            change_volume       1.00      0.80      0.89        30
                  routing       1.00      1.00      1.00        30
                      mpg       0.65      1.00      0.79     

In [324]:
sentences = [
  "NLP is fun",
  "are you "
]

pred_tokens = map(tokenizer.tokenize, sentences)
pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))

pred_token_ids = map(lambda tids: tids +[0]*(data.max_seq_len-len(tids)),pred_token_ids)
pred_token_ids = np.array(list(pred_token_ids))

predictions = model.predict(pred_token_ids).argmax(axis=-1)

for text, label in zip(sentences, predictions):
  print("text:", text, "\nintent:", classes[label])
  print()

text: NLP is fun 
intent: yes

text: are you  
intent: are_you_a_bot

